In [6]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D

In [7]:
#url = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv"
url = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv"
df = pd.read_csv(url)
df2 = df.loc[(df["Country/Region"] == "Hungary"),'1/23/20':]
#df2.loc[:,'3/23/20']= [167];
df2 = df2.assign(next = [156]) 
arr = df2.to_numpy()
#arr = list(np.arange(60,df2.shape[1]+60))
#arr = arr[arr != 0]
print(arr.shape[1])
i = 0
b = []
while i < arr.shape[1]:
    temp = arr[0][i]-arr[0][i-1]
    if temp > 0:
        b.append(temp)
    i += 1
timeseries = np.array(b)
timeseries

86


array([ 1,  2,  1,  2,  1,  2,  1,  1,  2,  2,  1,  4,  1,  5,  6,  2,  4,
        9, 11,  8, 11,  8, 14, 10, 13, 12,  8, 14])

In [22]:
# univariate cnn example
# split a univariate sequence into samples
def split_sequence(sequence, n_steps):
	X, y = list(), list()
	for i in range(len(sequence)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the sequence
		if end_ix > len(sequence)-1:
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
		X.append(seq_x)
		y.append(seq_y)
	return np.array(X), np.array(y)

# define input sequence
#raw_seq = [10, 20, 30, 40, 50, 60, 70, 80, 90]
raw_seq = timeseries
# choose a number of time steps
n_steps = 7
# split into samples
X, y = split_sequence(raw_seq, n_steps)
# reshape from [samples, timesteps] into [samples, timesteps, features]
n_features = 1
X = X.reshape((X.shape[0], X.shape[1], n_features))
# define model
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(n_steps, n_features)))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit(X, y, epochs=2000, verbose=0)
# demonstrate prediction
x_input = timeseries[-n_steps:]
x_input = x_input.reshape((1, n_steps, n_features))
print (x_input)
yhat = model.predict(x_input, verbose=0)
print(yhat)

[[[ 8]
  [14]
  [10]
  [13]
  [12]
  [ 8]
  [14]]]
[[17.641928]]


array([12,  8, 14])